# Including 'Hofstadter SP with Theta Phases' and 'First Band Approximation' Functions

In [1]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Theta Space.ipynb")
@nbinclude(joinpath("..","First Band Approximation Functions.ipynb"));

# Initial Parameters

In [8]:
Nx=4;Ny=4;N=Nx*Ny;p=1;q=Ny;PN=2;cut_off=5;T_size=10;

# Single Particle

In [9]:
function Sp_Theta_Op(Nx, Ny, alpha, Tx, Ty)
    
    N = Nx*Ny
    
    H_T = HSP_T(Nx, Ny, alpha, Tx, Ty, 0)
    
    basis = NLevelBasis(N) 
    
    H = SparseOperator(basis)
    
    for m in 1:N
        for n in 1:N
            H = H + H_T[m,n] * transition(basis, m, n)
        end
    end
    
    return H
end

Sp_Theta_Op (generic function with 2 methods)

In [12]:
H1 = Sp_Theta_Op(Nx, Ny, p/q, 1, 1);

In [13]:
# ...